# **Projek Akhir Praktikum Data Science**

> Analisis Sentimen Mengenai Vaksin COVID-19 Di Indonesia Menggunakan Metode Naive Bayes Classifier dan NLP Pada Sosial Media Twitter

**Oleh Kelompok 2 :**
1. Hazlan Muhammad Qodri (123190080) @hzlnqodrey
2. Elisia Dwi Rahayu (123190062) @elisiadwirahayu
3. Shania Septika Inayasari (123190055) @shaniainayasari

**Penjelasan Projek :**

Adapun pada penelitian menekankan kepada sentimen masyarakat terhadap mengenai vaksin COVID-19. Proses analisisnya akan dilakukan berdasarkan tweet yang menyertakan tagar vaksin dan pencarian di twitter dengan keyword vaksin covid 19.

## **1. Scraping Data from Twitter**

In [ ]:
%pip install snscrape

In [ ]:
import pandas as pd
import numpy as np
import snscrape.modules.twitter as sntwitter
import re


#### Query

In [ ]:
# Get All Covid Sentiment Data from January 1st, 2020 until November 1st, 2022
query = "covid since:2020-01-01 until:2022-11-01 lang:id"
limit = 2000 # limit 50k rows

In [ ]:
tweets = []

for tweet in sntwitter.TwitterSearchScraper(query).get_items():
    if len(tweets) == limit:
        break
    else:
        tweets.append([
            tweet.date,
            tweet.username,
            tweet.content
        ])

filename = 'tweets_covid_dataset_2k_raw_noindex.csv'
tweets_df = pd.DataFrame(tweets, columns=['Tanggal', 'Username', 'Text'])
tweets_df.to_csv(filename, index=False)
print('Scraping has completed!')

## **2. Wrangling Data** (Preprocessing)

In [ ]:
%pip install tweet-preprocessor
%pip install textblob
%pip install wordcloud
%pip install nltk

In [318]:
import preprocessor as preproc
from textblob import TextBlob
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import csv
import string

In [320]:
# get data from dataset
data = pd.read_csv('https://raw.githubusercontent.com/hzlnqodrey/projek-akhir-prak-ds-sentimen-analisis-twitter-covid19-nlp-bayes/main/data_csv/tweets_covid_dataset_2k_raw_noindex.csv')

In [321]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Tanggal   2000 non-null   object
 1   Username  2000 non-null   object
 2   Text      2000 non-null   object
dtypes: object(3)
memory usage: 47.0+ KB


In [322]:
data.sample(n=5)

,Tanggal,Username,Text
1223,2022-10-31 09:04:38+00:00,mxrshit_,"@milanicki_ @fidyfir @sembangggg @abdulhadiawang Sekarang zaman inflasi, beza zaman PH tak inflasi teruk, malah Covid 19 dialu-alukan. Dan sekarang akibatnya kita yang merasa."
1743,2022-10-31 05:42:56+00:00,respatimodjo,"@cebong_tolol @dr_koko28 @MayoClinic Sebelum ada vaksin covid juga orang2 pada meninggal dadakan. Orang ngerokok sebal sebul 2 bungkus per hari, cuma jaman itu belum ada yg dijadiin kambing item. Untung ada covid jadi vaksinnya bisa disalahin"
1107,2022-10-31 09:49:34+00:00,ffyiEXO,Reason i bought an Iphone sebabnya bila g concert kot la the boys ambik phone i and rakam mcm nie…pastu covid berpanjangan 🙃 whos the clown? Me! https://t.co/3rWemVL2d9
493,2022-10-31 14:23:26+00:00,hanamama_cos,"@juliejellies amiin julieee untuk saat ini belum berani soalnya tiap event asa belom kondusif, udahannya drama; terus masih suka liat temen2 kena covid jadi aku menghindari dulu deh yg bahaya2; udah susah2 biar ga jadi sarjana covid soalnya x'D"
305,2022-10-31 16:02:46+00:00,luokiodinson,"@BmkgSobat @Yucupsss @dr_koko28 @MayoClinic udah jelas vaksin itu cairan surga, ga mungkin lah jadi penyebab beberapa kejadian akhir2 ini, walaupun sebelum ada vaksin covid ga pernah ada kejadian2 seperti itu"


##### 1. Case Folding

In [323]:
data['Text'] = data['Text'].str.lower()

In [324]:
print('Case Folding Result : \n')
data.sample(n=5)

Case Folding Result : 



,Tanggal,Username,Text
129,2022-10-31 19:52:53+00:00,oktaaaaaaa___,@bebalboy @catchmeupid covid belum selesai masih ada varian baru + kasusnya banyak tp emg udah ga terlalu diberitakan. dan tiap tahun tanpa diramal juga banyak bencana wkwk
1772,2022-10-31 05:35:12+00:00,SAME_LITTLE88,@muhsupriyono93 @jjxnlee @bwfscore kekalahan di jo...gak bakal batalin gelar xd terbaik saat ini...top 4 siapa sih yang bisa juara back to back sampai sebanyak itu...dr akhir 2017 udah cetar...cuma kejeda covid doank
941,2022-10-31 10:52:54+00:00,BNPB_Indonesia,update infografis percepatan penanganan covid-19 di indonesia per tanggal 27 oktober 2022 pukul 12.00 wib.\n\n#bersatulawancovid19 https://t.co/vjxhcqhupi
1625,2022-10-31 06:36:03+00:00,jawapos,harga minyak mentah dunia turun imbas pengetatan covid-19 di tiongkok https://t.co/0zfzgtssdp
277,2022-10-31 16:16:02+00:00,farhanhers,oktober banyak lika liku nya. ditutup dengan kena covid~


In [325]:
data.isnull().sum()

Tanggal     0
Username    0
Text        0
dtype: int64

##### 2. Cleaning

In [326]:
# cleaning overall
def preprocessing_data(x):
    return preproc.clean(x)

data['Text'] = data['Text'].apply(preprocessing_data)


In [327]:
print('Cleaning Result : \n')
data.sample(n=5)

Cleaning Result : 



,Tanggal,Username,Text
412,2022-10-31 14:56:20+00:00,muslikin_,"lu di twit, gak keliatan.di ig, gak keliatan.kemaren abis covid.gimana gua gak curiga"
602,2022-10-31 13:21:18+00:00,BerkatPenyayang,stay safe everyone. rumours mengatakan jangkitan covid akan meningkat i am one of them just symptoms
104,2022-10-31 21:45:09+00:00,ladyfromnowher3,pas covid lg tinggi nya
1305,2022-10-31 08:29:51+00:00,maipo_kpts,macam mana korang boleh survive? kesan post covid ni sangat menyeksakan penat melampau tak pernah penat macam ni padahal baru je jalan sikit. ya allah syukurnya ada kereta. since dah start busy kat kampus baru terasa sangat.
778,2022-10-31 11:58:56+00:00,bckepri,"-pso bc tipe a tg. balai karimun, dan kpp pratama tg. balai karimun. bertempat di lapangan kanwil djbc khusus kepulauan riau, upacara digelar secara offline setelah dua tahun sebelumnya dilakukan secara daring akibat pandemi covid-19."


In [329]:
# cleaning remove_comments_special
def remove_comments_special(text):
    # remove tab, new line, and back slice
    text = text.replace('\\t', " ").replace('\\n', " ").replace(
        '\\u', " ").replace('\\', " ").replace('.', " ")
    # remove non ASCII (emoticon, chinese word, .etc)
    text = text.encode('ascii', 'replace').decode('ascii')
    # remove mention, link, hashtag
    text = ' '.join(
        re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)", " ", text).split())
    # remove ascii decoded
    text = ' '.join(
        re.sub("amp; ", " ", text).split())
    text = ' '.join(
        re.sub("lt; ", " ", text).split())
    text = ' '.join(
        re.sub("gt; ", " ", text).split())
    # remove single char
    text = ' '.join(
        re.sub(r"\b[a-zA-Z]\b", " ", text).split())
    return text

data['Text'] = data['Text'].apply(remove_comments_special)

# remove symbol
def remove_symbol(text):
    text = ''.join(
        re.sub(r"[\!\@\#\$\%\^\&\*\(\)\?\,\"\|\:]+", "", text)
    )
    return text

data['Text'] = data['Text'].apply(remove_symbol)

# remove punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans("", "", string.punctuation))


print('Cleaning Result : \n')
data.sample(n=5)

Cleaning Result : 



,Tanggal,Username,Text
1443,2022-10-31 07:39:35+00:00,SavarDian,proyek menghidupkan orang mati di china
639,2022-10-31 13:02:23+00:00,thelivingtr,secara hakikat tak blh kita berkata demikian sprmn tk blh kata covid menjangkiti yakni scr hakikat kerana sabda rasulullah tiada penyakit berjangkit tetapi tidak bermakna tidak boleh menyebut covid menjangkiti secara sebab penerangan lebih
1676,2022-10-31 06:13:12+00:00,88kising,saya belum pernah covid bu tapi jantung saya terasa lemah kalau dipeluk mbanya jam non-stop sampe alarm berdering
816,2022-10-31 11:44:32+00:00,21BeritaTerkini,malaysia temukan empat kasus covid-19 subvarian omicron xbb
1062,2022-10-31 10:05:59+00:00,GenBeWise,iyaa deipada nanti dia ngumumini lg covid


In [330]:
data.head(50)

,Tanggal,Username,Text
0,2022-10-31 23:59:58+00:00,lordkuyang,dulu di barat upn ada warung kayuh bambai bumbu habangnya sungguh banjar banar semenjak tahun ke covid mereka tutup padahal itu warung yang sangat mengobati kerinduan sama masakan banjar nasi kuning dan soto banjarnya juara sekali
1,2022-10-31 23:58:49+00:00,s_h_y_l_l_a,segala sakit yg disalahin vaksin covid lieur
2,2022-10-31 23:58:14+00:00,kunsh72,bukankah kamu dulu bagian rezim namun setelah terdepak dan sakit hati karena rezekinya hilang maka jadi sok kritis dulu mengkritisi vaksin covid namun saat divaksin pamer di medsos ucapanmu tidak konsisten bro semua karena sakit hati atas hilangnya rezeki
3,2022-10-31 23:57:58+00:00,erni076,aku kan ganti lama dulu sebelum covid menyerang sdh lama beda tahun beda hrga mas
4,2022-10-31 23:57:38+00:00,KENTUSIAS,trus masku kena covid lagi kek yah udah endemi masi ae
5,2022-10-31 23:55:03+00:00,jpnncom,covid-19 subvarian xbb dilaporkan telah masuk ke bali jakarta dan sejumlah wilayah lain di indonesia tetapi kabar dari kemenkes bikin lega
6,2022-10-31 23:53:17+00:00,Zyanpatra,covid-19 subvarian xbb masuk bali kabar dari kemenkes bikinlega
7,2022-10-31 23:52:42+00:00,republikaonline,pemerintah kerajaan arab saudi menyatakan masih mewajibkan vaksinasi meningitis bagi jamaah umrah indonesia yang akan pergi ke tanah suci pencabutan persyaratan kesehatan bagi jamaah umrah hanya berlaku bagi vaksinasi covid-19
8,2022-10-31 23:52:20+00:00,a_dlinaa,sakit tekak level covid ritu pun tak sakit mcmni
9,2022-10-31 23:51:57+00:00,AlbertSolo2,menteri kesehatan ri meminta masyarakat untuk mewaspadai penularan covid-19 pada awal tahun apalagi dengan masuknya virus omicron varian xbb di indonesia beberapa waktu lalu indonesia masih menjadi salah satu negara dengan jumlah kasus covid-19 tertinggi di dunia


##### 3. Tokenizing

In [331]:
def tokenize_data(text):
    return word_tokenize(text)

data['Text_Clean'] = data['Text'].apply(tokenize_data)

In [332]:
print('Tokenizing Result : \n')
data.sample(n=5)

Tokenizing Result : 



,Tanggal,Username,Text,Text_Clean
957,2022-10-31 10:47:33+00:00,drupadawan,ato bisa jadi penyebab nya penyakit covid itu sendiriklo baca2 di situs2 asing salah satu dampak buruk dari long covid ada di ginjal,"[ato, bisa, jadi, penyebab, nya, penyakit, covid, itu, sendiriklo, baca2, di, situs2, asing, salah, satu, dampak, buruk, dari, long, covid, ada, di, ginjal]"
820,2022-10-31 11:43:27+00:00,AkulahNazmi,banyak janji palsu tapi habuk pon takda kali janji pernah nak hantar pi uk nak hantar pi us dimaafkan sebab covid peginya tak pon kerja dah appointed dik kita buat macam2 dah transition bagai habuk pon takda raise pon sama dengan org lain sedih dik non,"[banyak, janji, palsu, tapi, habuk, pon, takda, kali, janji, pernah, nak, hantar, pi, uk, nak, hantar, pi, us, dimaafkan, sebab, covid, peginya, tak, pon, kerja, dah, appointed, dik, kita, buat, macam2, dah, transition, bagai, habuk, pon, takda, raise, pon, sama, dengan, org, lain, sedih, dik, non]"
414,2022-10-31 14:55:26+00:00,DanteAruma_,vilhelm parfumerie ini masuk kategori niche karena dia spesialis membuat produk parfum aja vp ini masih baru banget ya di dunia perwewangian karena butik pertamanya buka juni kasian juga ya plan bisnisnya diacak2 covid-19,"[vilhelm, parfumerie, ini, masuk, kategori, niche, karena, dia, spesialis, membuat, produk, parfum, aja, vp, ini, masih, baru, banget, ya, di, dunia, perwewangian, karena, butik, pertamanya, buka, juni, kasian, juga, ya, plan, bisnisnya, diacak2, covid-19]"
1482,2022-10-31 07:25:39+00:00,utusanborneo,malaysia berdepan gelombang kecil covid-19 susulan varian xbb,"[malaysia, berdepan, gelombang, kecil, covid-19, susulan, varian, xbb]"
1372,2022-10-31 07:59:16+00:00,Juna401,ya iyalah orang yg miskin dan nganggur udah pada gugur karena covid dan kelaparan,"[ya, iyalah, orang, yg, miskin, dan, nganggur, udah, pada, gugur, karena, covid, dan, kelaparan]"


##### 4. Filtering

In [333]:
# Filtering | Singkatan Indo
normalizad_word = pd.read_csv(
    "https://raw.githubusercontent.com/meisaputri21/Indonesian-Twitter-Emotion-Dataset/master/kamus_singkatan.csv", sep=";", header=None)
normalizad_word_dict = {}

for index, row in normalizad_word.iterrows():
    if row[0] not in normalizad_word_dict:
        normalizad_word_dict[row[0]] = row[1]


def normalized_term(document):
    return [normalizad_word_dict[term] if term in normalizad_word_dict else term for term in document]


data['Text_Clean'] = data['Text_Clean'].apply(normalized_term)
data.head(5)

,Tanggal,Username,Text,Text_Clean
0,2022-10-31 23:59:58+00:00,lordkuyang,dulu di barat upn ada warung kayuh bambai bumbu habangnya sungguh banjar banar semenjak tahun ke covid mereka tutup padahal itu warung yang sangat mengobati kerinduan sama masakan banjar nasi kuning dan soto banjarnya juara sekali,"[dulu, di, barat, upn, ada, warung, kayuh, bambai, bumbu, habangnya, sungguh, banjar, banar, semenjak, tahun, ke, covid, mereka, tutup, padahal, itu, warung, yang, sangat, mengobati, kerinduan, sama, masakan, banjar, nasi, kuning, dan, soto, banjarnya, juara, sekali]"
1,2022-10-31 23:58:49+00:00,s_h_y_l_l_a,segala sakit yg disalahin vaksin covid lieur,"[segala, sakit, yang , disalahin, vaksin, covid, lieur]"
2,2022-10-31 23:58:14+00:00,kunsh72,bukankah kamu dulu bagian rezim namun setelah terdepak dan sakit hati karena rezekinya hilang maka jadi sok kritis dulu mengkritisi vaksin covid namun saat divaksin pamer di medsos ucapanmu tidak konsisten bro semua karena sakit hati atas hilangnya rezeki,"[bukankah, kamu, dulu, bagian, rezim, namun, setelah, terdepak, dan, sakit, hati, karena, rezekinya, hilang, maka, jadi, sok, kritis, dulu, mengkritisi, vaksin, covid, namun, saat, divaksin, pamer, di, medsos, ucapanmu, tidak, konsisten, bro, semua, karena, sakit, hati, atas, hilangnya, rezeki]"
3,2022-10-31 23:57:58+00:00,erni076,aku kan ganti lama dulu sebelum covid menyerang sdh lama beda tahun beda hrga mas,"[saya, kan, ganti, lama, dulu, sebelum, covid, menyerang, sudah , lama, beda, tahun, beda, hrga, mas]"
4,2022-10-31 23:57:38+00:00,KENTUSIAS,trus masku kena covid lagi kek yah udah endemi masi ae,"[terus , masku, kena, covid, lagi, kek, yah, sudah , endemi, masi, ae]"


In [334]:
# Filtering | Stop Word
list_stopwords = (['yang', 'untuk', 'pada', 'ke', 'para', 'namun', 'menurut', 'antara', 'seperti', 'jika', 'jika', 'sehingga', 'mungkin', 'kembali', 'dan', 'ini', 'karena', 'oleh', 'saat', 'sekitar', 'bagi', 'serta', 'di', 'dari', 'sebagai', 'hal', 'ketika', 'adalah', 'itu', 'dalam', 'bahwa', 'atau', 'dengan', 'akan', 'juga', 'kalau', 'ada', 'terhadap', 'secara', 'agar', 'lain', 'jadi', 'yang ', 'sudah', 'sudah begitu', 'mengapa', 'kenapa', 'yaitu', 'yakni', 'daripada', 'itulah', 'lagi', 'maka', 'tentang', 'demi', 'dimana', 'kemana', 'pula', 'sambil', 'sebelum', 'sesudah', 'supaya', 'guna', 'kah', 'pun', 'sampai', 'sedangkan', 'selagi',
                  'sementara', 'tetapi', 'apakah', 'sebab', 'selain', 'seolah', 'seraya', 'seterusnya', 'dsb', 'dst', 'dll', 'dahulu', 'dulunya', 'anu', 'demikian', 'tapi', 'juga', 'mari', 'nanti', 'melainkan', 'oh', 'ok', 'sebetulnya', 'setiap', 'sesuatu', 'pasti', 'saja', 'toh', 'ya', 'walau', 'apalagi', 'bagaimanapun', 'yg', 'dg', 'rt', 'dgn', 'ny', 'd', 'klo', 'kalo', 'amp', 'biar', 'bikin', 'bilang', 'krn', 'nya', 'nih', 'sih', 'ah', 'ssh', 'om', 'ah', 'si', 'tau', 'tuh', 'utk', 'ya', 'cek', 'jd', 'aja', 't', 'nyg', 'hehe', 'pen', 'nan', 'loh', 'rt', '&amp', 'yah', 'ni', 'ret', 'za', 'nak', 'haa', 'zaa', 'maa', 'lg', 'eh', 'hmm', 'kali'])

list_stopwords = set(list_stopwords)


def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords]


data['Text_Clean'] = data['Text_Clean'].apply(stopwords_removal)
data.head(5)

,Tanggal,Username,Text,Text_Clean
0,2022-10-31 23:59:58+00:00,lordkuyang,dulu di barat upn ada warung kayuh bambai bumbu habangnya sungguh banjar banar semenjak tahun ke covid mereka tutup padahal itu warung yang sangat mengobati kerinduan sama masakan banjar nasi kuning dan soto banjarnya juara sekali,"[dulu, barat, upn, warung, kayuh, bambai, bumbu, habangnya, sungguh, banjar, banar, semenjak, tahun, covid, mereka, tutup, padahal, warung, sangat, mengobati, kerinduan, sama, masakan, banjar, nasi, kuning, soto, banjarnya, juara, sekali]"
1,2022-10-31 23:58:49+00:00,s_h_y_l_l_a,segala sakit yg disalahin vaksin covid lieur,"[segala, sakit, disalahin, vaksin, covid, lieur]"
2,2022-10-31 23:58:14+00:00,kunsh72,bukankah kamu dulu bagian rezim namun setelah terdepak dan sakit hati karena rezekinya hilang maka jadi sok kritis dulu mengkritisi vaksin covid namun saat divaksin pamer di medsos ucapanmu tidak konsisten bro semua karena sakit hati atas hilangnya rezeki,"[bukankah, kamu, dulu, bagian, rezim, setelah, terdepak, sakit, hati, rezekinya, hilang, sok, kritis, dulu, mengkritisi, vaksin, covid, divaksin, pamer, medsos, ucapanmu, tidak, konsisten, bro, semua, sakit, hati, atas, hilangnya, rezeki]"
3,2022-10-31 23:57:58+00:00,erni076,aku kan ganti lama dulu sebelum covid menyerang sdh lama beda tahun beda hrga mas,"[saya, kan, ganti, lama, dulu, covid, menyerang, sudah , lama, beda, tahun, beda, hrga, mas]"
4,2022-10-31 23:57:38+00:00,KENTUSIAS,trus masku kena covid lagi kek yah udah endemi masi ae,"[terus , masku, kena, covid, kek, sudah , endemi, masi, ae]"


## **2.1 Menejermah Tweet Clean hasil Preproc ke Bahasa Inggris** (Translating)

In [ ]:
%pip install googletrans==4.0.0rc1

In [ ]:
from googletrans import Translator

translate = Translator()

out = translate.translate(['veritas lux mea', 'lorem ipsum', 'gnocchi', 'dolor sit amet'], dest='en')

print(out.text)

## **2.2 Stemming**

## **3. Modelling Data** (Modeling and Training)

## **4. Klasifikasi Data dengan Naive Bayes** (Classification)